# Advanced Forex Trading System

## Features
- Multi-timeframe (H1, H4, D)
- Hyperparameter optimization
- Cross-validation
- Multiple currency pairs

## Install Dependencies

In [ ]:
!pip install oandapyV20 pandas numpy scikit-learn ta joblib optuna

## Import Libraries

In [ ]:
import pandas as pd
import numpy as np
from oandapyV20 import API
import oandapyV20.endpoints.instruments as instruments
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import optuna
from ta.trend import EMAIndicator, MACD
from ta.volatility import BollingerBands
from ta.momentum import RSIIndicator
import datetime
import joblib
import matplotlib.pyplot as plt

## Model Class Definition

In [ ]:
class AdvancedForexModel:
    def __init__(self, api_key):
        self.pairs = ['EUR_USD', 'GBP_USD', 'AUD_USD', 'USD_JPY']
        self.timeframes = ['H1', 'H4', 'D']
        self.api = API(access_token=api_key)
        self.models = {}
        self.best_params = {}
        self.performance_metrics = {}

## Data Collection Methods

In [ ]:
def fetch_multi_timeframe_data(self, pair):
    data = {}
    for tf in self.timeframes:
        params = {"count": 5000, "granularity": tf}
        r = instruments.InstrumentsCandles(instrument=pair, params=params)
        self.api.request(r)
        
        df = pd.DataFrame([
            {
                'time': candle['time'],
                'open': float(candle['mid']['o']),
                'high': float(candle['mid']['h']),
                'low': float(candle['mid']['l']),
                'close': float(candle['mid']['c']),
                'volume': float(candle['volume'])
            } for candle in r.response['candles']
        ])
        data[tf] = df
    return data

## Technical Analysis

In [ ]:
def add_technical_indicators(self, df):
    # Trend indicators
    df['ema_20'] = EMAIndicator(close=df['close'], window=20).ema_indicator()
    df['ema_50'] = EMAIndicator(close=df['close'], window=50).ema_indicator()
    macd = MACD(close=df['close'])
    df['macd'] = macd.macd()
    df['macd_signal'] = macd.macd_signal()
    
    # Volatility
    bb = BollingerBands(close=df['close'])
    df['bb_upper'] = bb.bollinger_hband()
    df['bb_lower'] = bb.bollinger_lband()
    
    # Momentum
    df['rsi'] = RSIIndicator(close=df['close']).rsi()
    
    return df

## Model Optimization

In [ ]:
def optimize_hyperparameters(self, X, y):
    def objective(trial):
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
            'max_depth': trial.suggest_int('max_depth', 10, 100),
            'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
            'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10)
        }
        
        tscv = TimeSeriesSplit(n_splits=5)
        scores = []
        
        for train_idx, val_idx in tscv.split(X):
            X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
            y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
            
            model = RandomForestClassifier(**params, random_state=42)
            model.fit(X_train, y_train)
            score = model.score(X_val, y_val)
            scores.append(score)
            
        return np.mean(scores)
    
    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=50)
    return study.best_params

## Training Pipeline

In [ ]:
def train_models(self):
    for pair in self.pairs:
        print(f"Training models for {pair}")
        data = self.fetch_multi_timeframe_data(pair)
        
        for timeframe in self.timeframes:
            df = self.add_technical_indicators(data[timeframe])
            X, y = self.prepare_training_data(df)
            
            X = X.dropna()
            y = y[X.index]
            
            best_params = self.optimize_hyperparameters(X, y)
            self.best_params[f"{pair}_{timeframe}"] = best_params
            
            model = RandomForestClassifier(**best_params, random_state=42)
            model.fit(X, y)
            
            self.models[f"{pair}_{timeframe}"] = model

## Signal Generation

In [ ]:
def generate_signals(self):
    signals = []
    for pair in self.pairs:
        data = self.fetch_multi_timeframe_data(pair)
        pair_signals = {}
        
        for timeframe in self.timeframes:
            df = self.add_technical_indicators(data[timeframe])
            X = df[self.feature_columns].iloc[-1:]
            
            model = self.models[f"{pair}_{timeframe}"]
            prediction = model.predict(X)
            probability = model.predict_proba(X)
            
            pair_signals[timeframe] = {
                'prediction': prediction[0],
                'confidence': np.max(probability[0])
            }
        
        consensus = self.calculate_consensus(pair_signals)
        if consensus['signal'] != 0:
            signals.append(self.format_signal(pair, consensus, data))
    
    return signals

## Model Usage Example

In [ ]:
# Initialize and train
api_key = 'YOUR_OANDA_API_KEY'
trader = AdvancedForexModel(api_key)
trader.train_models()

# Generate signals
signals = trader.generate_signals()
print("Trading Signals:")
for signal in signals:
    print(f"\n{signal}")